In [16]:
import keras
from keras.losses import SparseCategoricalCrossentropy
from keras.models import load_model
import keras.layers as L
import os, csv
import numpy as np
import scipy.io as sio
import pandas as pd

In [17]:
def load_data(record_path, delineation_path):
    data = sio.loadmat(record_path)
    signal = data['ECG']['data'][0][0]
    delineation = np.load(delineation_path) / 5

    signal = signal.T
    delineation = delineation.T
    signal_list = []
    delineation_list = []

    for i in range(0, signal.shape[0] - 2500, 4500):
        signal_slice = signal[i :i  + 4500, :]
        if signal_slice.shape[0] < 4500:
            signal_slice = signal[-4500:, :]

        delineation_slice = delineation[i : i + 4500, :]
        if delineation_slice.shape[0] < 4500:
            delineation_slice = delineation[-4500:, :]

        signal_list.append(signal_slice)
        delineation_list.append(delineation_slice)
    return signal_list, delineation_list
    # return signal.T, delineation.T

data_and_delin_path = 'DATA'
data_path = os.path.join(data_and_delin_path, "validation_set")
data_list = os.listdir(data_path)
delin_path = os.path.join(data_and_delin_path,"delineation_leads_val")
delin_list = os.listdir(delin_path)
if 'REFERENCE.csv' in data_list:
    data_list.remove('REFERENCE.csv')
if '.DS_Store' in data_list:
    data_list.remove('.DS_Store')
if 'REFERENCE.csv' in delin_list:
    delin_list.remove('REFERENCE.csv')
if '.DS_Store' in delin_list:
    delin_list.remove('.DS_Store')
data_list.sort()
delin_list.sort()

x_train = []
y_train = []
reference_dict = {}
reference_path = os.path.join(data_path, 'REFERENCE.csv')
reader = pd.read_csv(reference_path)
for i, row in reader.iterrows():
    reference_dict[row['Recording']] = int(row['First_label'])

dataset = []
for data, delin in zip(data_list, delin_list):
    cur_data_path = os.path.join(data_path, data)
    cur_delin_path = os.path.join(delin_path, delin)
    signal_list, delineation_list = load_data(cur_data_path, cur_delin_path)
    name = data.split('.')[0]
    reference_value = reference_dict[name] - 1
    for cur_signal, cur_delin in zip(signal_list, delineation_list):
        dataset.append([name, reference_value, np.concatenate((cur_signal, cur_delin), axis=1)])


X = np.array([x[2] for x in dataset])
y = np.array([x[1] for x in dataset])
name = np.array([x[0] for x in dataset])

np.save('X', X)
np.save('y', y)
np.save('name', name)